# My LEEDS 2023

Prepares data for the My LEEDS 2023 page 

In [ ]:
import os
import sys
import pandas as pd
import numpy as np

In [ ]:
lib_dir = os.path.abspath('../../../../lib/')
if not lib_dir in sys.path: sys.path.append(lib_dir)

import util.geo
import util.convert

Read in event data and filter by past / current (starting before today) tagged with the appropriate project name and event type

In [ ]:
data = pd.read_csv('../../../../data/metrics/events/master/all.csv', parse_dates=['start_date'])
data = data.loc[
    (data.project_type == 'Produce (Signature)') &
    (data.project_name == '12 - My LEEDS 2023') &
    (data.event_type.str.contains('Public Event -')) &
    (data.start_date < pd.Timestamp.now())
]

Calculate series of wards

In [ ]:
wards = data['ward_(from_venue)'].fillna(value='[]').pipe(util.convert.literal_converter).apply(np.unique)

Print a list of events which have no ward, or more than one

In [ ]:
data[wards.apply(len) != 1]

Pick the first ward in the list and create a new column

In [ ]:
data['ward_code'] = wards.apply(lambda x: x[0] if len(x) > 0 else np.nan).pipe(util.geo.normalise_leeds_wards).pipe(util.geo.ward_name_to_code)

Calculate audience size

In [ ]:
data['audience_size'] = data.actual_audience_size_number_of_participants_in_person.fillna(0) + data.actual_audience_size_number_of_participants_online.fillna(0)

Create output directory

In [ ]:
OUT_DIR = '../../../../docs/metrics/events/my-leeds-2023/_data/events'
os.makedirs(OUT_DIR, exist_ok=True)

Create counts of events by ward and save CSV

In [ ]:
by_ward = pd.DataFrame(
  {
    'events': data.groupby('ward_code').audience_size.count(),
    'audience': data.groupby('ward_code').audience_size.sum(),
  }
)
by_ward.to_csv(os.path.join(OUT_DIR, 'by_ward.csv'))

Create summary by week

In [ ]:
by_week = pd.DataFrame({
  'events': data.groupby('start_date').audience_size.count(),
  'audience': data.groupby('start_date').audience_size.sum().astype(int)
}).resample('W-FRI').sum()
by_week.index.name = 'week_ending'
by_week['cumulative_events'] = by_week.events.cumsum()
by_week['cumulative_audience'] = by_week.audience.cumsum()
by_week.to_csv(os.path.join(OUT_DIR, 'by_week.csv'))

Create summary by month

In [ ]:
by_month = by_week.resample('M').sum()
by_month['cumulative_events'] = by_month.events.cumsum()
by_month['cumulative_audience'] = by_month.audience.cumsum()
by_month.index.name = 'month_ending'
by_month.to_csv(os.path.join(OUT_DIR, 'by_month.csv'))

Create headline counts of events

In [ ]:
headlines = by_month.drop(
    columns=['events', 'audience']
).rename(
    columns=lambda n: n.replace('cumulative', 'total')
).iloc[-1]
headlines['ward_count'] = len(by_ward.index)

headlines.to_json(os.path.join(OUT_DIR, 'headlines.json'))